In [1]:
!pip install langchain_community
!pip install InstructorEmbedding
!pip install sentence-transformers==2.2.2
!pip install pip install pypdf
!pip install chromadb
!pip install langchain_ollama

ERROR: Could not find a version that satisfies the requirement install (from versions: none)
ERROR: No matching distribution found for install


In [2]:
import os
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
#import dotenv
from tqdm import tqdm
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader

## To run Hugging Face OpenSource models
# Needs to manually install Visual C++ Tools from: https://visualstudio.microsoft.com/visual-cpp-build-tools/
from InstructorEmbedding import INSTRUCTOR
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

c:\Users\USER\OneDrive\maestria\Proyecto Integrador\proyecto_integrador_equipo_27\.venv\Lib\site-packages\InstructorEmbedding\instructor.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


In [3]:
import torch
import torch.nn as nn

# Check if CUDA is available
print(f"CUDA Available: {torch.cuda.is_available()}")

# Print CUDA device name
if torch.cuda.is_available():
    print(f"Device Name: {torch.cuda.get_device_name(0)}")

CUDA Available: True
Device Name: NVIDIA GeForce RTX 3050


In [4]:
# Ensure GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


### 2.- Chunking
Is the process of breaking down a large input text into smaller pieces.
This ensures that the text fits the input size of the embedding model and improves retrieval efficiency.

In [5]:
# Define the directory containing the PDF files
pdf_directory = r"../WebScrap_PDFs"

# Define the persistent directory containing the VectorDB
persistent_dir= "VectorDB/HS_codes/model_WebScrapping_PDFs"

# WARNING! :Only runs with this version
###### !pip install sentence-transformers==2.2.2  ######
#Define the sentence-transformer model:
#For English
#sentence-transformers/LaBSE
#embed_model = "sentence-transformers/all-MiniLM-L6-v2"
embed_model = "sentence-transformers/all-mpnet-base-v2"
#embed_model = "w601sxs/b1ade-embed"
#embed_model = "jinaai/jina-embeddings-v3"
#embed_model = "WhereIsAI/UAE-Large-V1"

#embed_model = "nvidia/NV-Embed-v2" #Needs sentence transfomer pip upgrade
#embed_model = "dunzhang/stella_en_1.5B_v5"
#embed_model = "Salesforce/SFR-Embedding-2_R"
#embed_model = "Alibaba-NLP/gte-large-en-v1.5"


#For Spanish 
#projecte-aina/aguila-7b
#embed_model = "hiiamsid/sentence_similarity_spanish_es"

#Other sentence-transformer settings
model_kwargs = {'device': 'cuda:0'}  # specify GPU device
encode_kwargs = {'normalize_embeddings': True}

# Initialize an empty list to store all the extracted documents
all_documents = []

# Iterate over all files in the directory
for filename in os.listdir(pdf_directory):
    # Check if the file is a PDF
    if filename.endswith(".pdf"):
        # Create the full file path
        file_path = os.path.join(pdf_directory, filename)
        
        # Load the PDF using PyPDFLoader
        loader = PyPDFLoader(file_path)
        
        # Load the documents from the PDF
        documents = loader.load()
        
        # Add the documents to the list of all_documents
        all_documents.extend(documents)

# Print the number of documents extracted
print(f"Total number of documents extracted: {len(all_documents)}")

Total number of documents extracted: 429


In [6]:
print(all_documents[0])

page_content=' 
 
Section  I 
 
LIVE ANIMALS; ANIMAL PRODUCTS  
 
Notes.  
1.- Any reference in this Section  to a particular genus or species of an animal, except where the context 
otherwise requires, includes a reference to the young of that genus or species.  
2.- Except where th e context otherwise requires, throughout the Nomenclature any reference to “dried” 
products also covers products which have been dehydrated, evaporated or freeze -dried.  
_________________  ' metadata={'source': '../WebScrap_PDFs\\0100_2022e_Section Notes.pdf', 'page': 0}


In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=30, chunk_overlap=3)
chunks = text_splitter.split_documents(all_documents)
#print(chunks)

### 3.- Embedding

A technique for representing text data as numerical vectors, which can be input into machine learning models. The embedding model is responsible for converting text into these vectors.

In [8]:
#Only runs with this version
#!pip install sentence-transformers==2.2.2

hf_embed_model = HuggingFaceInstructEmbeddings(
    model_name=embed_model,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

load INSTRUCTOR_Transformer
max_seq_length  512


### 4.- Vector DB

A collection of pre-computed vector representations of text data for fast retrieval and similarity search, with capabilities like CRUD operations, metadata filtering, and horizontal scaling.

In [9]:
#With progress bar workaround
vector_db = None
with tqdm(total=len(chunks), desc="Creating embeddings...") as pbar:
    for d in chunks:
        if vector_db:
            vector_db.add_documents([d])
        else:
            #When no GPU is available
            #vector_db = Chroma.from_documents([d],embed_model, persist_directory=persistent_dir )
            
            #To enable embeddings running on GPU
            vector_db = Chroma.from_documents([d],hf_embed_model, persist_directory=persistent_dir)
        pbar.update(1)

Creating embeddings...: 100%|██████████| 34429/34429 [18:33<00:00, 30.91it/s]   


### Simple test over Vector DB

This response would contain the answer to that question, this needs to be sent to the LLM to paraphrase and extract only the needed info

In [10]:
question = "What is an HS code? "
docs = vector_db.similarity_search(question)
print(docs)

[Document(metadata={'page': 0, 'source': '../WebScrap_PDFs\\1582_2022e_Tools, implements, cutlery, spoons and forks, of base metal; parts thereof of base metal.pdf'}, page_content='Heading  H.S.  \nCode'), Document(metadata={'page': 2, 'source': '../WebScrap_PDFs\\1582_2022e_Tools, implements, cutlery, spoons and forks, of base metal; parts thereof of base metal.pdf'}, page_content='Heading  H.S.  \nCode'), Document(metadata={'page': 4, 'source': '../WebScrap_PDFs\\1684_2022e_Nuclear reactors, boilers, machinery and mechanical appliances; parts thereof.pdf'}, page_content='Heading  H.S.  \nCode'), Document(metadata={'page': 10, 'source': '../WebScrap_PDFs\\1684_2022e_Nuclear reactors, boilers, machinery and mechanical appliances; parts thereof.pdf'}, page_content='Heading  H.S.  \nCode')]


### Q&A Chain

This will run peristantly, consulting already stored vectors

### Using a LLM Chain to provide context and system prompts on every query

In [12]:
#from langchain_chroma import Chroma
from langchain_community.vectorstores import Chroma

#For some reason, "context" cant be used as input variable, it should be named as "summaries"
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate

##### Embedding model (sentence-transformer)
model_name = embed_model
model_kwargs = {'device': 'cuda:0'}  # specify GPU device
encode_kwargs = {'normalize_embeddings': True}
hf_embed_model = HuggingFaceInstructEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

##### LLM model
# loading the Llama3 model from local
llm_model = OllamaLLM(model="llama3.1",
                temperature=0.2,
                num_thread=8,
                )
# loading the vectorstore
vectorstore = Chroma(persist_directory=persistent_dir, embedding_function=hf_embed_model)
# casting  the vectorstore as the retriever, taking the best 3 similarities
retriever = vectorstore.as_retriever(search_kwargs={"k":5})

# formating the docs
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

template = """Always answer in English. 
If you can infer an specific code (based on a very good match) you must respond only with this kind fo format "XXXX.XX" and answer it at the beggining with: 
"The suggested HS code is: " 

In the case you can't find a matching code for the description, respond with:
'Your inquiry is very subjective, to provide an accurate HS Code I need you to be more specific, please include size, weight, origin, meaning of use, etc. 
So far, I can narrow it to this chapters/parts : 'provide 3 possible codes".

context:
{summaries}

Question:
{question}
"""

# Define the LLM chain (using the Llama3.1 model)
llm_chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm_model,
    chain_type='stuff',
    retriever=retriever,
    return_source_documents=True,  # To get both the answer and source docs
    chain_type_kwargs={
            "prompt": PromptTemplate(
                template=template,
                #For some reason, "context" cant be used as input variable, it should be named as "summaries"
                input_variables=["question", "summaries"],
            ),
        },
)

context = """As a logistics shipping arrival inspector, your primary responsibility is to inspect incoming shipments and accurately classify goods 
using the Harmonized System (HS) code based on the descriptions provided in the shipping manifests. You will thoroughly review the manifest details, 
including product type, material composition, function, and intended use, to determine the correct HS code. 

Your task is to:
Carefully read and analyze the product descriptions from the manifest.
Identify key characteristics of the goods, such as 
type (e.g., electronics, textiles, machinery), 
material (e.g., plastic, metal, organic), 
and usage (e.g., household, industrial, medical).
Use your knowledge of the HS code classification system to assign the most appropriate HS code for each product based on its description.
Ensure compliance with international trade regulations by selecting precise codes to avoid delays or penalties.
Remember to be thorough and accurate in your classification, as this impacts customs processing, tariffs, and legal requirements."""

#-----Benchmark query------
#Response should be: "0106.19" 
#query = "Live Dog"
query = "A Live Dog of breed Schnauzer from Germany"
#query = "Live Pig"
#query = "Live Buffalo"
#query = "What HS Code belongs to a wrench?"
#query = "What HS Code belongs to a live dog?"
#query = "What HS Code belongs to:'CERAMIC TABLEWARE  2688 BOXES PO 5 39548'?"
#query = "Covers for cellphone screen"

# Execute the chain with the query
#For some reason, "context" cant be used as input variable, it should be named as "summaries"
result = llm_chain({"question": query, "summaries": context})

# Process the results
#print(result.keys())
print("The response of the LLM is: ", result["answer"])

#print("The documents content used for this response are:")
#for i in range(len(result["source_documents"])):
#    print(result["source_documents"][i].page_content)

     
#results = vectorstore.similarity_search_with_score(query=query, k=5)
## Print similarity results
#for doc, score in results:
#    print(f"Document content: {doc.page_content}, Similarity Score: {score}")



load INSTRUCTOR_Transformer
max_seq_length  512
The response of the LLM is:  The suggested HS code is: 0103.10
